# Implementasi Agglomerative

### Pembacaan Data

In [120]:
import pandas as pnd
import heapq
from collections import Counter
from sklearn.metrics import confusion_matrix

iris = pnd.read_csv('iris.csv')
#source: https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv

### Pemisahan Data Training dan Label

In [121]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

data_train = np.array(iris.iloc[:, 0:4])
iris_label = iris.iloc[:,-1]
species_encoder = LabelEncoder().fit(iris_label)
iris_label_encoded = species_encoder.transform(iris_label)

### Implementasi Kelas Agglomerative

In [122]:
class AgglomerativeClustering():

    def __init__(self, cluster_num=2, linkage="single"):
        self.cluster_num = cluster_num
        self.linkage = linkage
        self.data = None
        self.data_num = None
        self.col_num = None
        self.label = None
        self.dm = None
        self.sda = None

    def distance(self,i1,i2):
        dist = 0
        for attr in range(self.col_num):
            dist += abs(self.data[i1][attr] - self.data[i2][attr])
        return dist

    def populate_distance_matrix(self):
        for i in range(self.data_num):
            self.label.append(i)

        for i1 in range(self.data_num):
            for i2 in range(self.data_num):
                self.dm[i1][i2] = self.distance(i1,i2)

    def process_shortest_path(self):
        for i in range(self.data_num):
            j = 0
            min = 99999
            self.sda[i][0] = min
            self.sda[i][1] = i
            self.sda[i][2] = j
            while j < self.data_num:
                if(self.dm[i][j] < min and self.dm[i][j] != 0):
                    min = self.dm[i][j]
                    self.sda[i][0] = min
                    self.sda[i][1] = i
                    self.sda[i][2] = j
                j += 1

    def single_linkage(self,id1,id2):
        for y in range(self.data_num):
            if(self.dm[id1][y] < self.dm[id2][y]):
                self.dm[id2][y] = self.dm[id1][y]
            else:
                self.dm[id1][y] = self.dm[id2][y]

        for x in range(self.data_num):
            if(self.dm[x][id1] < self.dm[x][id2]):
                self.dm[x][id2] = self.dm[x][id1]
            else:
                self.dm[x][id1] = self.dm[x][id2]

    def complete_linkage(self,id1,id2):
        for y in range(self.data_num):
            if(self.dm[id1][y] and self.dm[id2][y]):
                if(self.dm[id1][y] > self.dm[id2][y]):
                    self.dm[id2][y] = self.dm[id1][y]
                else:
                    self.dm[id1][y] = self.dm[id2][y]

        for x in range(self.data_num):
            if(self.dm[x][id1] and self.dm[x][id2]):
                if(self.dm[x][id1] > self.dm[x][id2]):
                    self.dm[x][id2] = self.dm[x][id1]
                else:
                    self.dm[x][id1] = self.dm[x][id2]

    def average_linkage(self,id1,id2):
        for y in range(self.data_num):
            if(self.dm[id1][y] and self.dm[id2][y]):
                average = self.dm[id2][y] + self.dm[id1][y] / 2
                self.dm[id2][y] = average
                self.dm[id1][y] = average

        for x in range(self.data_num):
            if(self.dm[x][id1] and self.dm[x][id2]):
                average = self.dm[x][id1] + self.dm[x][id2] / 2
                self.dm[x][id2] = average
                self.dm[x][id1] = average

    def update_distance_matrix(self,id1,id2):
        if self.linkage == "single":
            self.single_linkage(id1,id2)
        elif self.linkage == "complete":
            self.complete_linkage(id1,id2)
        else:
            self.average_linkage(id1,id2)

    def is_outlier_big(self):
        uniqueL = list(set(self.label))
        unsortedL = Counter(self.label)
        val = []

        for i in range(len(uniqueL)):
            val.append(unsortedL[uniqueL[i]])

        outlier = self.data_num - sum(heapq.nlargest(self.cluster_num, val))

        return (float(outlier)/self.data_num >= 0.03)

    def merging(self):
        self.process_shortest_path()

        min = self.sda[0][0]
        id1 = self.sda[0][1]
        id2 = self.sda[0][2]
        for i in range(self.data_num):
            if(self.sda[i][0] < min):
                min = self.sda[i][0]
                id1 = self.sda[i][1]
                id2 = self.sda[i][2]
        id1 = id1.astype(np.int64)
        id2 = id2.astype(np.int64)
        if(id1 < id2):
            self.label[:] = [id1 if x==id2 else x for x in self.label]
        else:
            self.label[:] = [id2 if x==id1 else x for x in self.label]
        self.dm[id1][id2] = 0
        self.dm[id2][id1] = 0

        self.update_distance_matrix(id1,id2)

        cn = len(list(set(self.label)))

        if self.cluster_num != cn and self.is_outlier_big():
            self.merging()

    def fit(self, data_train):
        self.data = data_train
        self.data_num = len(self.data)
        self.col_num = len(self.data[0])
        self.label = []
        self.dm = np.zeros(shape=(self.data_num,self.data_num))
        self.sda = np.zeros(shape=(self.data_num,3))
        
        self.populate_distance_matrix()
        self.merging()

        cluster = []
        result = Counter(self.label).most_common(self.cluster_num)
        for i in range(self.cluster_num):
            cluster.append(result[i][0])

        for i in range(self.data_num):
            if not self.label[i] in cluster:
                self.label[i] = -1

        return self

### Percobaan

In [128]:
clf = AgglomerativeClustering(3,"single")
agglomerative = clf.fit(data_train)
mat = confusion_matrix(agglomerative.label, iris_label_encoded)

pnd.crosstab(iris_label, np.array(agglomerative.label))

col_0,-1,0,3,50
species,,,,
setosa,1,48,1,0
versicolor,0,0,0,50
virginica,3,0,0,47


In [129]:
purity = float(mat[1].max() + mat[2].max() + mat[5].max()) / float(mat.sum())

print("Purity: ", purity)
print mat

('Purity: ', 0.66)
[[ 0  1  0  3  0  0]
 [ 0 48  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  1  0  0  0  0]
 [ 0  0 50 47  0  0]]


### Perbandingan dengan Agglomerative dari Sklearn

In [125]:
from sklearn.cluster import AgglomerativeClustering as SklearnAgglomerative

agglomerative = SklearnAgglomerative(n_clusters=3).fit(data_train)
mat = confusion_matrix(agglomerative.labels_, iris_label_encoded)

pnd.crosstab(iris_label, agglomerative.labels_)

col_0,0,1,2
species,,,
setosa,0,50,0
versicolor,49,0,1
virginica,15,0,35


In [127]:
purity = float(mat[0].max() + mat[1].max() + mat[2].max()) / float(mat.sum())

print("Purity: ", purity)

('Purity: ', 0.8933333333333333)
[[ 0 49 15]
 [50  0  0]
 [ 0  1 35]]
